In [1]:
import ikpy
import ikpy.chain
import ikpy.utils.plot as plot_utils

import numpy as np
import time
import math

# import ipywidgets as widgets
# import serial


In [2]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from mpl_toolkits.mplot3d import Axes3D
import numpy as np

# Instead of ikpy.utils.plot, create your own plotting functions
def plot_robot_configuration(ax, joint_positions, joint_orientations=None):
    """Plot robot configuration in 3D"""
    # Extract x, y, z positions
    x = [pos[0] for pos in joint_positions]
    y = [pos[1] for pos in joint_positions]
    z = [pos[2] for pos in joint_positions]
    
    # Plot the robot links
    ax.plot(x, y, z, 'b-o', linewidth=2, markersize=8, label='Robot Links')
    
    # Plot joint positions
    ax.scatter(x, y, z, c='red', s=100, label='Joints')
    
    # Plot end effector
    ax.scatter(x[-1], y[-1], z[-1], c='green', s=150, label='End Effector')
    
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.legend()
    ax.grid(True)

In [15]:
urdf_file = "../urdf/xarm_v2_ik_final.urdf"
urdf_file = "../urdf/xarm_v3_ik.urdf"
my_chain = ikpy.chain.Chain.from_urdf_file(
    urdf_file,
    active_links_mask=[False, True, True, True, True, True, False],
    base_elements=["xarm_base_link"]
)

In [16]:
def degs_to_input(x):
    #x = 0 to 500, x = -135 to 0 and x = 135 to 
    x = max(-134.99, min(x, 134.99)) 
    return int(500.*x/135. + 500.)

def degs_to_radians(x):
    return x * math.pi / 180.0


def scale_angles(angles, min_val=-2.35619, max_val=2.35619, out_max=1000):
    arr = np.array(angles, dtype=float)
    scaled = (arr - min_val) / (max_val - min_val) * out_max
    scaled = np.clip(scaled, 0, out_max)
    return scaled.astype(int).tolist()

In [ ]:

from scipy.spatial.transform import Rotation as R
import numpy as np

oritent_axis = "Z"

z_orientation = np.array([0.0, 0.0, -1.0])  # world down, as a numpy vector

target_position = [0.0, 0.1, 0.1]
target_orientation = np.array(target_position) * np.array([1.0, 1.0, -1.0])  # world down, as a numpy vector
# target_orientation = np.array([0.5, 0.0, -0.5])  # world down, as a numpy vector

ik = my_chain.inverse_kinematics(
    target_position=target_position,
    target_orientation=target_orientation,
    orientation_mode=oritent_axis,
    # initial_position=[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
)

target_position = [0.2, 0.0, 0.0]

ik = my_chain.inverse_kinematics(
    target_position=target_position,
    target_orientation=target_orientation,
    orientation_mode=oritent_axis, 
    initial_position=ik
)

target_position = [0.2, 0.0, -0.1]
target_orientation = target_orientation + z_orientation #np.array([0.5, 0.25, -5.0])  # world down, as a numpy vector

ik = my_chain.inverse_kinematics(
    target_position=target_position,
    target_orientation=target_orientation,
    orientation_mode=oritent_axis, 
    initial_position=ik
)

# target_position = [0.0, 0.2, -0.1]
target_position = [0.0, 0.0, 0.6]
target_orientation = target_orientation + z_orientation #np.array([0.5, 0.25, -5.0])  # world down, as a numpy vector
target_orientation = np.array([0.0, 0.0, 0.0])

ik = my_chain.inverse_kinematics(
    target_position=target_position,
    target_orientation=target_orientation,
    orientation_mode=oritent_axis, 
    initial_position=ik
)


In [26]:
print(ik)
joint_pos = ik.tolist()

print("The angles of each joints are : ", joint_pos)

computed_position = my_chain.forward_kinematics(ik)
print("Computed position: %s, original position : %s" % (computed_position[:3, 3], target_position))
print("Computed position (readable) : %s" % [ '%.2f' % elem for elem in computed_position[:3, 3] ])

angles = joint_pos#[:-1]
angles[3] = -angles[3]  # Invert the 4th joint angle

final_angles = scale_angles(angles)
print("robot inputs: ", final_angles)
#robot inputs:  [500, 500, 162, 384, 296, 500, 500]

[ 0.         -1.56676657 -1.59202291  0.52286705 -1.02532013  0.
  0.        ]
The angles of each joints are :  [0.0, -1.566766571306174, -1.5920229141218851, 0.522867046676786, -1.025320126627976, 0.0, 0.0]
Computed position: [ 0.17569471  0.00070801 -0.08527784], original position : [0.2, 0.0, -0.1]
Computed position (readable) : ['0.18', '0.00', '-0.09']
robot inputs:  [500, 167, 162, 389, 282, 500, 500]


In [7]:
%matplotlib widget
#%matplotlib widget
import matplotlib.pyplot as plt
fig, ax = plot_utils.init_3d_figure()
fig.set_figheight(9)  
fig.set_figwidth(13)  
my_chain.plot(ik, ax, target=target_position)
plt.xlim(-0.5, 0.5)
plt.ylim(-0.5, 0.5)
ax.set_zlim(0, 0.6)
plt.ion()

ModuleNotFoundError: No module named 'ipympl'

[500, 500, 498, 499, 501, 500]


In [6]:
# import sys
# sys.path.append("workspace/xarm_stuff/xarm_remote_control/")
from bus_servo_serial import BusServoSerial


In [14]:
servo = BusServoSerial('/dev/ttyUSB1')
servo.run(5,int(405))

SerialException: [Errno 16] could not open port /dev/ttyUSB1: [Errno 16] Device or resource busy: '/dev/ttyUSB1'

In [19]:
servo.set_positions(final_angles, 1000)


In [6]:
# servo.run_mult((final_angles), 1000)
servo.set_positions([500]*6, 1000)

In [ ]:
import serial
con = serial.Serial('/dev/ttyUSB0', baudrate=115200, timeout=0.01)
con.write(b'\x03')
con.write(b'bus_servo.set_positions([500,0,500,500,500,500],1000)\r\n')


55

In [ ]:
import time
for i in range(2):
    con.write(b'bus_servo.set_positions([500,1000,500,500,500,500],1000)\r\n')
    # con.flushOutput()
    time.sleep(1)
    res = con.write(b'bus_servo.get_positions()\r\n')
    print(con.read(1000))
    time.sleep(1)
    con.write(b'bus_servo.set_positions([500,0,500,500,500,500],1000)\r\n')

b'et_positions([500,0,500,500,500,500],1000)\r\n>>> bus_servo.set_positions([500,1000,500,500,500,500],1000)\r\n>>> bus_servo.get_positions()\r\n[False, 904, 501, 504, 501, 498]\r\n>>> bus_servo.set_positions([500,0,500,500,500,500],1000)\r\n>>> bus_servo.set_positions([500,1000,500,500,500,500],1000)\r\n>>> bus_servo.get_positions()\r\n'
b'[493, 848, 501, 504, 501, 499]\r\n>>> bus_servo.set_positions([500,0,500,500,500,500],1000)\r\n>>> bus_servo.set_positions([500,1000,500,500,500,500],1000)\r\n>>> bus_s'


: 

In [ ]:
res = con.write(b'bus_servo.get_positions()\r\n')
con.read(1000)

b'\r\n>>> bus_servo.set_positions([500,500,500,500,500,500],1000)\r\n>>> bus_servo.get_positions()\r\n[511, 498, 499, 498, 496, 498]\r\n>>> '

: 

In [3]:
res[1]

TypeError: 'int' object is not subscriptable